# ¡¡¡ ANTES DE COMENZAR CAMBIA EL NOMBRE DE ESTE ARCHIVO !!!

Cambia el nombre del archivo de modo que empiece con las tres siguientes letras `P1-` y vaya seguido del nombre del alumno  Para los espacios en blanco usar guiones bajos `_`

Ejemplo para un alumno:

`P1-Don_Quijote.pynb`


## Instalación de librerías necesarias

In [1]:
#Si te da error esta linea de código en colab prueba con la siguiente linea comentada en lugar de la primera
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!apt-get update openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark

### Cargamos el entorno

In [9]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"
import findspark
findspark.init()

### Importamos pyspark y creamos una sessión

In [10]:
from pyspark.sql import SparkSession
ss = SparkSession.builder.master("local[*]").getOrCreate()

In [38]:
logsRDD = ss.sparkContext.textFile("/content/apachelog.txt")
logsRDD.takeSample(False, 10)

['sparky.jvnc.net;-;-;1995-08-10 22:18:47;GET;/facilities/lc39a.html;HTTP/1.0;200;7008',
 'tonya.ita.lgc.com;-;-;1995-08-04 15:41:27;GET;/images/NASA-logosmall.gif;HTTP/1.0;200;786',
 '129.173.4.90;-;-;1995-08-13 11:20:27;GET;/images/whatsnew.gif;HTTP/1.0;200;651',
 'is502.tech.chiba-u.ac.jp;-;-;1995-08-04 06:33:04;GET;/;HTTP/1.0;200;7034',
 'brick.bpa.gov;-;-;1995-08-04 21:21:32;GET;/images/ksclogosmall.gif;HTTP/1.0;200;3635',
 'underdog.jpl.nasa.gov;-;-;1995-08-21 15:16:30;GET;/history/gemini/gemini-vii/gemini-vii.html;HTTP/1.0;200;2487',
 'globus.ffi.no;-;-;1995-08-16 09:27:47;GET;/shuttle/resources/orbiters/endeavour-logo.gif;HTTP/1.0;200;5052',
 'hella.stm.it;-;-;1995-08-16 03:27:10;GET;/shuttle/missions/sts-69/images/KSC-95EC-0760.gif;HTTP/1.0;200;66162',
 '198.109.36.15;-;-;1995-08-13 20:25:21;GET;/images/USA-logosmall.gif;HTTP/1.0;200;234',
 'cliffy.lfwc.lockheed.com;-;-;1995-08-16 11:12:54;GET;/images/kscmap-tiny.gif;HTTP/1.0;200;2537']

### Para cada ejercicio, es obligatorio almacenar el resultado en las variables definidas en cada TODO
1. ¿Cuántas lineas se han registrado en los logs?

Para este ejercicio el resultado debe ser almacenado en `uno`

In [39]:
#TODO
# Contar el número de líneas
uno = logsRDD.count()

#Muestra el resultado
uno

977769

2. ¿Obten las diferentes números de campo que tiene cada linea de los log y el recuento de lineas para cada uno de los casos?

Resultado del tipo `[(numero de campos, recuento de apariciones),...]`:

`[(6, 10), (9, 321), (1, 554), (10, 15)]`

Para este ejercicio el `RDD` resultante debe ser almacenado en `dos`

In [40]:
#TODO
# Contar el número de campos de cada línea y el recuento de líneas para cada caso
dos = logsRDD.map(lambda line: (len(line.split(";")), 1)) \
                          .reduceByKey(lambda x, y: x + y)

#Muestra el resultado
dos.collect()

[(9, 964781), (1, 12986), (10, 1), (11, 1)]

3. Elimina aquellos logs que tengan longitud diferente a 9 y guardalo en un nuevo RDD llamado `cleanLogsRDD`.
A partir de este ejercicio, se utilizará el RDD llamado `cleanLogsRDD` que no cuenta con datos corruptos.

Deberían salirte $964781$ lineas

In [41]:
#TODO
# Filtrar los logs que tienen longitud diferente de 9
cleanLogsRDD = logsRDD.filter(lambda line: len(line.split(";")) == 9)


# Contar el número de líneas en cleanLogsRDD
test = cleanLogsRDD.count()

#Muestra el resultado
print(test)

964781


4. Tipos de peticiones realizadas al servidor y frecuencia absoluta

Resultado del tipo `[(tipo de peticion, recuento de apariciones),...]`:

`[(GET, 10), (POST, 321)]`

Para este ejercicio el `RDD` resultante debe ser almacenado en `cuatro`

In [42]:
#TODO
# Contar el número de peticiones GET y POST
get_count = cleanLogsRDD.filter(lambda line: line.split(";")[4] == 'GET').count()
post_count = cleanLogsRDD.filter(lambda line: line.split(";")[4] == 'POST').count()

# Crear el RDD con el resultado
cuatro = ss.sparkContext.parallelize([('GET', get_count), ('POST', post_count)])

#Muestra el resultado
cuatro.collect()

[('GET', 962318), ('POST', 68)]

5. Para peticiones de tipo GET, obtener los código de respuestas producidos y sus frecuencias absolutas.

Resultado del tipo `[(codigo de respuesta, recuento de apariciones),...]`:

`[(404, 10), (200, 321)]`

Para este ejercicio el `RDD` resultante debe ser almacenado en `cinco`

In [43]:
#TODO
# Filtrar peticiones de tipo GET y obtener códigos de respuesta y sus frecuencias absolutas
cinco = cleanLogsRDD.filter(lambda line: line.split(";")[4] == 'GET') \
                                 .map(lambda line: (int(line.split(";")[7]), 1)) \
                                 .reduceByKey(lambda x, y: x + y)

#Muestra el resultado
cinco.collect()

[(304, 74774), (403, 47), (200, 879376), (302, 2292), (404, 5827), (500, 2)]

6. Para peticiones de tipo GET, obtener los código de respuestas producidos y sus frecuencias absolutas pero esta vez por familias (2XX, 3XX, 4XX, 5XX)


Resultado del tipo `[(familia de codigo de respuesta, recuento de apariciones),...]`:

`[(4XX, 10), (2XX, 321)]`

Para este ejercicio el rdd resultante debe ser almacenado en `seis`

In [44]:
#TODO
# Función para clasificar los códigos de respuesta en familias
def clasificar_codigo_respuesta(codigo_respuesta):
    if codigo_respuesta >= 200 and codigo_respuesta < 300:
        return '2XX'
    elif codigo_respuesta >= 300 and codigo_respuesta < 400:
        return '3XX'
    elif codigo_respuesta >= 400 and codigo_respuesta < 500:
        return '4XX'
    elif codigo_respuesta >= 500 and codigo_respuesta < 600:
        return '5XX'
    else:
        return 'Otros'

# Obtener códigos de respuesta producidos para peticiones GET y sus frecuencias absolutas
seis = cleanLogsRDD.filter(lambda linea: linea.split(";")[4] == 'GET') \
                                   .map(lambda linea: (clasificar_codigo_respuesta(int(linea.split(";")[7])), 1)) \
                                   .reduceByKey(lambda x, y: x + y)

#Muestra el resultado
seis.collect()

[('4XX', 5874), ('3XX', 77066), ('2XX', 879376), ('5XX', 2)]

7. Top ten host más visitados

Resultado del tipo `[(host, recuento de apariciones),...]`:

`[(google.com, 10), (yahoo.es, 321),...]`

Para este ejercicio el resultado debe ser almacenado en `siete`

In [45]:
#TODO
# Obtener los diez hosts más visitados y sus recuentos de apariciones
top_diez_hosts = cleanLogsRDD.map(lambda linea: (linea.split(";")[0], 1)) \
                             .reduceByKey(lambda x, y: x + y) \
                             .sortBy(lambda x: x[1], ascending=False) \
                             .take(10)

# Crear el RDD con el resultado
siete = ss.sparkContext.parallelize(top_diez_hosts).collect()

#Muestra el resultado
siete

[('edams.ksc.nasa.gov', 3737),
 ('piweba5y.prodigy.com', 3067),
 ('piweba4y.prodigy.com', 2690),
 ('piweba3y.prodigy.com', 2658),
 ('www-d1.proxy.aol.com', 2591),
 ('news.ti.com', 2358),
 ('163.206.89.4', 2317),
 ('www-b2.proxy.aol.com', 2289),
 ('www-b3.proxy.aol.com', 2254),
 ('www-d2.proxy.aol.com', 2229)]

8. Top 10 urls visitadas no encontradas (código respuesta 404)

Resultado del tipo `[(url_completa, recuento de apariciones),...]`:

`[(google.com/robot.txt, 10), (yahoo.es/readme.txt, 321),...]`

Tened en cuenta que la url esta formada por dos campos del conjunto de datos, el host y el resto de la url.

Para este ejercicio el resultado debe ser almacenado en `ocho`

In [46]:
#TODO
# Obtener las 10 URL más visitadas no encontradas (código de respuesta 404) y sus recuentos de apariciones
top_diez_urls_404 = cleanLogsRDD.filter(lambda linea: linea.split(";")[7] == '404') \
                                .map(lambda linea: (linea.split(";")[0] + linea.split(";")[5], 1)) \
                                .reduceByKey(lambda x, y: x + y) \
                                .sortBy(lambda x: x[1], ascending=False) \
                                .take(10)

# Crear el RDD con el resultado
ocho = ss.sparkContext.parallelize(top_diez_urls_404).collect()

#Muestra el resultado
ocho

[('ts8-1.westwood.ts.ucla.edu/images/Nasa-logo.gif', 37),
 ('nexus.mlckew.edu.au/images/nasa-logo.gif', 34),
 ('203.13.168.17/images/nasa-logo.gif', 25),
 ('203.13.168.24/images/nasa-logo.gif', 25),
 ('onramp2-9.onr.com/images/nasa-logo.gif', 22),
 ('crl5.crl.com/images/nasa-logo.gif', 22),
 ('198.40.25.102.sap2.artic.edu/images/nasa-logo.gif', 21),
 ('msp1-16.nas.mr.net/images/nasa-logo.gif', 20),
 ('dial055.mbnet.mb.ca/images/nasa-logo.gif', 19),
 ('dialup551.chicago.mci.net/images/nasa-logo.gif', 18)]

9. Obtener el tamaño medio de los archivos descargados GET y subidos POST

Resultado del tipo `[(tipo de peticion, tamaño medio)]`:

`[(GET, 10), (POST, 321)]`

Para este ejercicio el `RDD` resultante debe ser almacenado en `nueve` y el valor tamaño medio debe ser un entero.

El resultado optimo se consigue operando en una sola linea de código.

In [47]:
#TODO
# Obtener el tamaño medio de los archivos para peticiones GET y POST
nueve = cleanLogsRDD.map(lambda linea: (linea.split(";")[4], int(linea.split(";")[8]))) \
                    .filter(lambda x: x[1] > 0) \
                    .groupByKey() \
                    .map(lambda x: (x[0], sum(x[1]) / len(x[1])) if len(x[1]) > 0 else (x[0], 0))

#Muestra el resultado
nueve.collect()

[('GET', 19605.648513379754), ('POST', 828.0217391304348)]